# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


### First, the data 
Download the data from [here](https://www.kaggle.com/ntnu-testimon/paysim1) and place it in the provided data folder. <br>
Import the dataset and produce some discriptive statistics and plots. What do you think will be the important features in determining the outcome?

In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Your code here
data = pd.read_csv('../data/data.csv')
data.head()

In [ ]:
# all types seem OK
data.dtypes

In [ ]:
data.describe().transpose()

In [ ]:
null_cols = data.isna().sum()
null_cols[null_cols > 0]

In [ ]:
# I'm guessing we will try to predict the "isFraud" column
# But its not very highly correlated to any other column, not even to "isFlaggedFraud"

# corr = data.corr()
# sns.heatmap(corr,annot=True)

data.corr()

### What is the distribution of the outcome? 

In [ ]:
# not sure which distribution they are asking for
data['isFraud'].value_counts()#.plot.bar()

data['isFlaggedFraud'].value_counts()#.plot.bar()

In [ ]:
# Your response here

#8213/6354407 is Fraud, but only 16/6362604 is flagged as Fraud

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [ ]:
# Your code here
# I don't see any time variable in this dataset?


### Run a logisitc regression classifier and evaluate its accuracy.

In [ ]:
data.drop(['type','nameOrig', 'nameDest'], axis=1, inplace=True)

# Your code here
from sklearn.model_selection import train_test_split
y = data['isFraud']
X = data.drop('isFraud', axis=1)  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
from sklearn.linear_model import LogisticRegression 
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
pd.DataFrame({'test':y_test, 'predicted':y_pred})

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_test, y_pred, normalize='true')

In [ ]:
accuracy_score(y_test, y_pred)

### Now pick a model of your choice and evaluate its accuracy.

In [ ]:
# Your code here
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 8)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
confusion_matrix(y_test, y_pred, normalize='true')

In [ ]:
accuracy_score(y_test, y_pred)

### Which model worked better and how do you know?

In [ ]:
# Your response here
# model k neighbors doesnt finish running :-( seems to be taking too long, more than an hour!